# TV Script Generation



This project is about generating your own Simpsons TV scripts using RNNs. You'll be using part of the Simpsons dataset of scripts from 27 seasons. The Neural Network built will generate a new TV script for a scene at Moe's Tavern.

## Get the Data

The data is pulled using a subset of the original dataset. It consists of only the scenes in Moe's Tavern. This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [4]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [7]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing. Implement the following preprocessing functions below:
* Lookup Table
* Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids. In this function, create two dictionaries:
* Dictionary to go from the words to an id, we'll call vocab_to_int
* Dictionary to go from the id to word, we'll call int_to_vocab
Return these dictionaries in the following tuple (vocab_to_int, int_to_vocab)

In [26]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    int_to_vocab = {ii: word for ii, word in enumerate(vocab, 1)}

    return (vocab_to_int, int_to_vocab)

# Testing create_lookup_tables
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters. However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".
Implement the function token_lookup to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". Create a dictionary for the following symbols where the symbol is the key and value is the token:
* Period ( . )
* Comma ( , )
* Quotation Mark ( " )
* Semicolon ( ; )
* Exclamation mark ( ! )
* Question mark ( ? )
* Left Parentheses ( ( )
* Right Parentheses ( ) )
* Dash ( -- )
* Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it. This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [29]:
def token_lookup():
    return {
        '.' : '||Period||',
        ',' : '||Comma||',
        '\"' : '||Quotation_Mark||',
        ';' : '||Semicolon||',
        '!' : '||Exclamation_Mark||',
        '?' : '||Question_Mark||',
        '(' : '||Left_Parentheses||',
        ')' : '||Right_Parentheses||',
        '--' : '||Dash||',
        '\n' : '||Return||'
    }

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [31]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [32]:
import helper
import numpy as np
import problem_unittests as test

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
* get_inputs
* get_init_cell
* get_embed
* build_rnn
* build_nn
* get_batches

### Check the Version of TensorFlow and Access to GPU

In [34]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/krypten/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the get_inputs() function to create TF Placeholders for the Neural Network. It should create the following placeholders:
* Input text placeholder named "input" using the TF Placeholder name parameter.
* Targets placeholder
* Learning Rate placeholder

Return the placeholders in the following tuple (Input, Targets, LearningRate)

In [44]:
def get_inputs():
    input = tf.placeholder(tf.float32, [None, None], name='input')
    targets = tf.placeholder(tf.float32, [None, None], name='target')
    learning_rate = tf.placeholder(tf.float32)
    
    return input, targets, learning_rate

tests.test_get_inputs(get_inputs)

Tests Passed


## Build RNN Cell and Initialize
Stack one or more BasicLSTMCells in a MultiRNNCell.
* The Rnn size should be set using rnn_size
* Initalize Cell State using the MultiRNNCell's zero_state() function
 * Apply the name "initial_state" to the initial state using tf.identity()

Return the cell and initial state in the following tuple (Cell, InitialState)

In [53]:
num_layers = 10

def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)        
        return lstm
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')

    return cell, initial_state

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to input_data using TensorFlow. Return the embedded sequence.

In [54]:
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the get_init_cell() function. Time to use the cell to create a RNN.
* Build the RNN using the tf.nn.dynamic_rnn()
 * Apply the name "final_state" to the final state using tf.identity()

Return the outputs and final_state state in the following tuple (Outputs, FinalState)

In [171]:
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state

tests.test_build_rnn(build_rnn)

Tests Passed
